In [ ]:
import geopandas as gp
import pandas as pd
import maup
import matplotlib.pyplot as plt
import os

In [ ]:
al_blocks = gp.read_file("/Users/peterhorton/Documents/RDH/raw_data/census/2020_TIGER_B/al_b_2020_bound/al_b_2020_bound.shp")


file_hold = gp.read_file("/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/gen/al_gen_22_prec/al_gen_22_no_splits_prec.shp")
file_hold.rename(columns = {"County_x":"County", "COUNTYFP_x":"COUNTYFP"}, inplace = True)


In [ ]:

def make_vf_points(vf):
    vf['lat'] = vf['lat'].astype(float)
    vf['lon'] = vf['lon'].astype(float)
    points = gp.GeoDataFrame(vf,geometry = gp.points_from_xy(vf.lon, vf.lat, crs='EPSG:4326'))
    assert(len(vf)==len(points))
    return points

vf_correspondence = pd.read_csv("./starting_vf_matches.csv")
vf_correspondence["County"] = vf_correspondence["ELEC_22_ID"].apply(lambda x:x.split("-:-")[0])

vf_correspondence_dict = dict(zip(vf_correspondence["VF_ID"],vf_correspondence["ELEC_22_ID"]))


holder = pd.read_csv("/Users/peterhorton/Downloads/AL_individual_file000.gz")
holder["UNIQUE_ID"] = holder["co_fips"].astype(str).str.zfill(3)+"-"+holder["precinct"]
holder["UNIQUE_ID"] = holder["UNIQUE_ID"].str.strip()
#holder = holder[(holder["gen_vote"]=="Y")]
holder["Count"] = 1
holder = make_vf_points(holder)

holder = holder.to_crs(file_hold.crs)
holder["ELEC_PREC_MATCH"] = holder["UNIQUE_ID"].map(vf_correspondence_dict).fillna(holder["UNIQUE_ID"])
holder["ELEC_PREC_MATCH"] = holder["ELEC_PREC_MATCH"].fillna("NO ASSIGNMENT")


holder_grouped = holder.groupby("UNIQUE_ID").sum()
holder_grouped.reset_index(inplace = True)



Completed:
- Crenshaw, line in VFD Honorville

To-Do:

- DeKalb, lines in MCKestes Comm Ctr, Fyfee Senior Ctr, Tenboreck Comm Ctr

- Mobile lines in Davidson High School, Abba Shrine, Hope Chapel AMEZ, JE Turner Elem, McDavid Jones Elem, The River, VIA_Senior Center, needs more area in Revelation Missionary, The Grand Hall, Three Circle CH



- Coffee has precinct shards, Pleasant Ridge VTG CTR messed up, perhaps duplicates? Note: Coffee I did create an "Unassigned"

- Washington, if you have time to look that would be helpful
- Clarke, downtown Jackson especially could use a look

In [ ]:
for county in list(file_hold["County"].unique()):
#for county in ["Jefferson"]:

    county_data = file_hold[file_hold["County"]==county].copy(deep = True)
    fips = county_data["COUNTYFP"].unique()[0]
    county_vf = holder[holder["co_fips"]==fips]
    county_join = gp.sjoin(county_data, county_vf, how = "right")

    if not os.path.exists("./vf_map_checks/"+county):
        os.mkdir("./vf_map_checks/"+county)
    for precinct in list(vf_correspondence[vf_correspondence["County"]==county]["ELEC_22_ID"].unique()):
        print(precinct)
        vf_id_list = list(vf_correspondence[vf_correspondence["ELEC_22_ID"]==precinct]["VF_ID"].unique())
        

        prec = file_hold[file_hold["UNIQUE_ID"]==precinct].copy(deep = True)
        ax = prec.boundary.plot(figsize = (10,10))
        holder[holder["UNIQUE_ID"].isin(vf_id_list)].plot(ax = ax, color = "red")
        fips = prec["COUNTYFP"].values[0]
        intersect = county_join[(county_join["UNIQUE_ID_x"]==precinct)&~(county_join["UNIQUE_ID_y"].isin(vf_id_list))]

   
        if intersect.shape[0] != 0:
            intersect.plot(ax = ax, column = "ELEC_PREC_MATCH", legend = True, legend_kwds = {'loc':(1,0)})
        
        
        
        
        ax.set_title(precinct)
#         if precinct!="Escambia-:-BETHEL ROBERTS":
        ax.figure.savefig("./vf_map_checks/"+county+"/"+precinct.replace("/","-")+".jpg", bbox_inches = "tight")


In [ ]:
def check_pber_implied_turnout(block_gdf, pber_file, output_path, elec_prefix):
    pber_file.index = pber_file["UNIQUE_ID"]
    block_gdf["UNIQUE_ID"] = maup.assign(block_gdf, pber_file)
    prec_census = block_gdf.groupby("UNIQUE_ID").sum()
    prec_census.reset_index(inplace = True, drop = False)
    prec_census["VAP_mod"] = prec_census["P0010001"] - prec_census["P0050003"]
    pber_file.reset_index(inplace = True, drop = True)
    joined_data = pd.merge(prec_census, pber_file, on = "UNIQUE_ID", how = "outer")
    joined_data["VAP_mod"] = joined_data["VAP_mod"].fillna(0)
    
    joined_data = get_estimated_total_votes(joined_data, elec_prefix)
    
    return joined_data
    
    #generate_analysis(joined_data, "COUNTYFP", output_path)
    
    
def get_estimated_total_votes(pber_file, prefix):
    cols = [i for i in list(pber_file.columns) if i[:len(prefix)]==prefix]
    pber_file["Tot_Votes"] = 0
    for col in cols:
        pber_file["Tot_Votes"] += pber_file[col]
    return pber_file

In [ ]:
joined = check_pber_implied_turnout(al_pl, file_hold, "./","G22USS")

In [ ]:
joined["Turnout_Est"] = joined["Tot_Votes"]/joined["VAP_mod"]

In [ ]:
joined[joined["Turnout_Est"]>1]

In [ ]:
plt.scatter(joined["VAP_mod"], joined["Tot_Votes"])
plt.xlabel("VAP mod of precinct's maup-ed blocks")
plt.ylabel("Total Number of Senator Votes")

#plt.scatter(range(0,10000), range(0,10000), s = 1)

In [ ]:
joined[(joined["VAP_mod"]>10000) & (joined["Tot_Votes"]<1000)]

In [ ]:
county = pd.read_csv("/Users/peterhorton/Documents/RDH/raw_data/census/PL_COUNTYFP_NAMES.csv")
al_cnty_fips_dict = dict((zip(county[county["STUSAB"]=="AL"]["NAME"],county[county["STUSAB"]=="AL"]["COUNTYFP"].astype(str).str.zfill(3))))

In [ ]:
al_cnty_fips_dict

In [ ]:
al_cnty_fips_dict_reversed = {value:key.replace(" County","") for key, value in al_cnty_fips_dict.items()}

In [ ]:
al_cnty_fips_dict_reversed

In [ ]:
holder[holder["co_fips"]==25].to_file("./clarke_vf.shp")

In [ ]:
for county in list(holder["co_fips"].unique()):
    ax = holder[holder["co_fips"]==county].plot(column = "precinct", legend = True, legend_kwds = {"loc":(1,-.25)}, cmap = "tab20c")
    file_hold[file_hold["COUNTYFP"]==county].boundary.plot(ax = ax, color = "black")
    ax.set_title(al_cnty_fips_dict_reversed[county.astype(str).zfill(3)])
    ax.figure.savefig("./map_checks/"+al_cnty_fips_dict_reversed[county.astype(str).zfill(3)]+".jpg")

In [ ]:
# DeKalb - blocks just not granular enough
# Choctaw - looks okay from Q
# Wilcox - looks fine from Q
Calhoun-:-HARVEST CHURCH OF GOD, needs to be expanded
    

In [ ]:
county = 11
ax = holder[holder["co_fips"]==county].plot(column = "precinct", legend = True, legend_kwds = {"loc":(1,-.25)}, cmap = "tab20c")
file_hold[file_hold["COUNTYFP"]==county].boundary.plot(ax = ax, color = "black")

## Bullock - Redrew to fix

In [ ]:
bullock_pl = al_pl[al_pl["COUNTYFP20"]=="011"].copy(deep = True)
bullock_precs = file_hold[file_hold["COUNTYFP"]==county].copy(deep = True)
bullock_precs.index = bullock_precs["UNIQUE_ID"]

bullock_pl["Assignment"] = maup.assign(bullock_pl, bullock_precs)

bullock_pl = bullock_pl[["GEOID20","Assignment"]].to_csv("./bullock_baf.csv", index = False)

In [ ]:
holder[holder["co_fips"]==county].to_file("./county_checks/bullock_vf.shp")
file_hold[file_hold["COUNTYFP"]==county].to_file("./county_checks/bullock_precs.shp")

In [ ]:
bullock_precincts = file_hold[file_hold["COUNTYFP"]==county].copy(deep = True)

In [ ]:
county = 15

ax = holder[holder["co_fips"]==county].plot(column = "precinct", legend = True, legend_kwds = {"loc":(1,-.25)}, cmap = "tab20c")
file_hold[file_hold["COUNTYFP"]==county].boundary.plot(ax = ax, color = "black")
file_hold[file_hold["UNIQUE_ID"]=="Calhoun-:-HARVEST CHURCH OF GOD"].boundary.plot( color = "yellow", ax =ax)
# holder[holder["co_fips"]==county].to_file("./county_checks/calhoun_vf.shp")
# file_hold[file_hold["COUNTYFP"]==county].to_file("./county_checks/calhoun_precs.shp")

## Calhoun

In [ ]:
county = 15
calhoun_pl = al_pl[al_pl["COUNTYFP20"]=="015"].copy(deep = True)
calhoun_precs = file_hold[file_hold["COUNTYFP"]==county].copy(deep = True)
calhoun_precs.index = calhoun_precs["UNIQUE_ID"]

calhoun_pl["Assignment"] = maup.assign(calhoun_pl, calhoun_precs)

calhoun_pl = calhoun_pl[["GEOID20","Assignment"]].to_csv("./calhoun_baf.csv", index = False)

In [ ]:
county = 23
ax = holder[holder["co_fips"]==county].plot(column = "precinct", legend = True, legend_kwds = {"loc":(1,-.25)}, cmap = "tab20c")
file_hold[file_hold["COUNTYFP"]==county].boundary.plot(ax = ax, color = "black")
file_hold[file_hold["UNIQUE_ID"]=="Choctaw-:-HURRICANE"].boundary.plot(ax = ax, color = "yellow")
holder[holder["co_fips"]==county].to_file("./county_checks/choctaw_vf.shp")
file_hold[file_hold["COUNTYFP"]==county].to_file("./county_checks/choctaw_precs.shp")

In [ ]:
holder[(holder["co_fips"]==county)&(holder["precinct"]=="0032-HURRICANE")]

In [ ]:
county = 49
ax = holder[holder["co_fips"]==county].plot(column = "precinct", legend = True, legend_kwds = {"loc":(1,-.25)}, cmap = "tab20c")
file_hold[file_hold["COUNTYFP"]==county].boundary.plot(ax = ax, color = "black")
file_hold[file_hold["UNIQUE_ID"]=="DeKalb-:-FIRE #3 E HIGHLAND D-1"].boundary.plot(ax = ax, color = "yellow")

In [ ]:
holder[holder["co_fips"]==county].to_file("./county_checks/dekalb_vf.shp")
file_hold[file_hold["COUNTYFP"]==county].to_file("./county_checks/dekalb_precs.shp")

In [ ]:
county = 131
holder[holder["co_fips"]==county].to_file("./county_checks/wilcox_vf.shp")
file_hold[file_hold["COUNTYFP"]==county].to_file("./county_checks/wilcox_precs.shp")

In [ ]:
county = 131
ax = holder[holder["co_fips"]==county].plot()
file_hold[file_hold["COUNTYFP"]==county].boundary.plot(ax = ax)
file_hold[file_hold["UNIQUE_ID"]=="Wilcox-:-ST PAUL AME CH"].boundary.plot(ax = ax, color = "yellow")


In [ ]:
holder_grouped[~holder_grouped["UNIQUE_ID"].isin(list(vf_correspondence["VF_ID"]))]

In [ ]:
holder_grouped[~holder_grouped["UNIQUE_ID"].isin(list(vf_correspondence["VF_ID"]))].to_csv("./remaining_unjoined_vf_precincts.csv", index = False)

In [ ]:
file_hold[~file_hold["UNIQUE_ID"].isin(list(vf_correspondence["ELEC_22_ID"]))]

In [ ]:
file_hold[~file_hold["UNIQUE_ID"].isin(list(vf_correspondence["ELEC_22_ID"]))].to_csv("./remaining_unjoined_elec_precincts.csv", index = False)

In [ ]:
full_list = ['Coffee',
 'Cullman',
 'Autauga',
 'Bullock',
 'Butler',
 'Pike',
 'Tuscaloosa',
 'Talladega',
 'St. Clair',
 'Morgan',
 'Montgomery',
 'Baldwin',
 'Blount',
 'Chilton',
 'Cleburne',
 'Crenshaw',
 'Dale',
 'DeKalb',
 'Elmore',
 'Etowah',
 'Fayette',
 'Greene',
 'Lauderdale',
 'Lee',
 'Limestone',
 'Madison',
 'Marengo',
 'Mobile',
 'Marshall',
 'Cherokee',
 'Dallas',
 'Monroe',
 'Walker',
 'Wilcox']

full_list.sort()

In [ ]:
al_votes = gp.read_file("/Users/peterhorton/Downloads/al_2022_gen_2020_vtds/al_2022_gen_2020_vtds.shp")

In [ ]:
import os

In [ ]:
counties_w_data = list(vf_correspondence["County"].unique())

In [ ]:
for county in counties_w_data:
    prec_count = file_hold[file_hold["County"]==county].shape[0]
    vf_relations_list = list(vf_correspondence[vf_correspondence["County"]==county]["ELEC_22_ID"].unique())
    vf_relations = len(vf_correspondence[vf_correspondence["County"]==county]["ELEC_22_ID"].unique())
    precinct_list = list(vf_correspondence[vf_correspondence["County"]==county]["VF_ID"].unique())
    
    
    if prec_count != vf_relations:
        fips = file_hold[(file_hold["County"]==county)]["COUNTYFP"].unique()[0]
        print(fips)
        print(list(file_hold[(file_hold["County"]==county)&~(file_hold["UNIQUE_ID"].isin(vf_relations_list))]["UNIQUE_ID"].values))
        print(holder[(holder["co_fips"]==fips)&(holder["gen_vote"]=="Y")&~(holder["UNIQUE_ID"].isin(precinct_list))]["UNIQUE_ID"].value_counts())
        print(list(holder[(holder["co_fips"]==fips)&(holder["gen_vote"]=="Y")&~(holder["UNIQUE_ID"].isin(precinct_list))]["UNIQUE_ID"].unique()))

In [ ]:
holder[holder["co_fips"].isin([25,129])].to_file("./clarke_washington_vf.shp")

In [ ]:
shape = gp.read_file("/Users/peterhorton/Documents/RDH/pber_collection/AL/2022/gen/big_join_one.shp")

In [ ]:
shape[(shape['County']=="Crenshaw")].plot(column = "UNIQUE_ID", legend = True, legend_kwds = {'loc':(1.01,0)})

In [ ]:
al_blocks = gp.read_file("/Users/peterhorton/Documents/RDH/raw_data/census/2020_TIGER_B/al_b_2020_bound.zip")

In [ ]:
gp.overlay()

In [ ]:
shape[(shape['County']=="Crenshaw")&(shape["UNIQUE_ID"].isin(["Crenshaw-:-VFD HONORVILLE","Crenshaw-:-RUTLEDGE COMM_ CTR"]))].plot(column = "UNIQUE_ID", legend = True, legend_kwds = {'loc':(1.01,0)})

Counties That Pass Voterfile Test
1. 'Autauga', - passes map test
2. 'Baldwin', - passes map test
3. 'Blount', - passes map test
4. 'Chilton', - passes map test
5. 'Cleburne', - passes map test
6. 'Cullman', - passes map test
7. 'Dale', - passes map test
8. 'Dallas', - passes map test
9. 'Elmore', - passes map test
10. 'Etowah', - passes map test
11. 'Fayette', - passes map test
12. 'Greene', - passes map test
13. 'Lauderdale', - passes map test
14. 'Lee', - passes map test
15. 'Marengo', - passes map test
16. 'Marshall', - passes map test
17. 'Monroe', - passes map test 
18. 'Montgomery', - passes map test
19. 'Morgan', - passes map test
20. 'Talladega', - passes map test
21. 'Tuscaloosa', - passes map test
22. 'Wilcox' - passes map test
23. Barbour-:-passes map test
24. Chambers-:-passes map test
25. Clay-:-passes map test
26. Conecuh-:-passes map test
27. Covington-:-passes map test
28. Hale-:- passes map test
29. Houston-:- passes map test
30. Marion-:- passes map test
31. Pickens-:- passes map test
32. Russell-:- passes map test
33. Tallapoosa -:- passes map test
34. Madison -:- passes map test
35. Franklin-:- passes map test, except voter file is not granular enough in places (Precinct 2)
36. Choctaw-:-passes map test, but voterfile looks sparse
37. Geneva-:- passes map test, but "GILMORE VTG HOUSE"
38. Lamar-:- passes map test
39. Lowndes-:- passes map test
40. Perry-:- passes map test, maybe minor issues
41. Winston- passes map test, maybe minor issues
42. Shelby-:- passes map test
43. Sumter-:- passes map test
44. Cherokee-:- passes map test, check voterfile counts
45. Bullock-:- passes map test, pretty close, might not be able to do it at block level
46. St. Clair-:- maybe Friendship Baptist Church not much
47. Colbert-:-Appears to pass map test, reversed VF
48. Coosa-:-passes map test
49. Escambia-:-passes map test, but not great voted or not data
50. Jefferson-:-5060 the only weird one, looks like a VF issue
51. Walker-:- passes map test, pretty close ["Thach School", "Manchester Comm Bldg"] (VF match off, 127-3001-ABUNDANT LIFE CHURCH split)
 
Cleaning Needed
- 'Butler', [Forest Home Voting House, American Legion, Harrison's VTG House, McKenzie City Hall, Searcy VFD, 
- 'Crenshaw', - passes map test, pretty close (Rutledge Comm Center)
- 'Coffee', ['Elba Church of Christ, Elba National Guard, Eprise State, Johns Chapel, Pleasant Ridge VTG, 
- 'DeKalb', [Fyfee Senior Ctr]

- 'Mobile', - ["Abba Shrine Temple", "Collin Rhodes Elem", "Cottage Hill Baptist Ch", "J E Turner Elem"


Cleaning Completed
- Bibb-:- ALTERNATIVE SCHOOL, GREENPOND FIRE DEPT_
- Calhoun-:-1ST UN_METHODIST
- Henry-:- ["GREATER SHILOH BAPT_", "HEADLAND VTG CTR_1"]
- Jackson-:-["SECTION NEW CITY HALL"]
- Macon-:- [NATIONAL GUARD ARMORY, SOJOURNER COMM CTR]
- Lawrence-:- ["MCCARLEY CTR_ 6-1","HILLSBORO CITY HALL 1-1","WOLF SPRING BAPT_", "PINE GROVE CH 36-1", "CHALYBEATE FIRE DEPT17-1"]]
- 'Pike',maybe Galloway Road Comm Ctr(?)

To Implement Changes
- 'Limestone', [ELKTON ROAD BAPTIST CH] - Spencer sent file

To Check

2. 
3. Washington-:-Closer look needed
4. Clarke-:-weird voterfile, join might be off


More VF work
5. Randolph-:-Can't make correspondence



In [ ]:
[33, 37, 53, 25, 111, 129]

In [ ]:
al_cnty_fips_dict

In [ ]:
raw_vf = pd.read_csv("/Users/peterhorton/Downloads/AL_individual_file000.gz")
raw_vf["UNIQUE_ID"] = raw_vf["co_fips"].astype(str).str.zfill(3)+"-"+raw_vf["precinct"]
raw_vf["UNIQUE_ID"] = raw_vf["UNIQUE_ID"].str.strip()

In [ ]:
remaining_counties = raw_vf[raw_vf["co_fips"].isin([73])]

In [ ]:
import numpy as np
remaining_counties["Count_Vote"] = np.where(remaining_counties["gen_vote"]=="Y",1,0)

In [ ]:
remaining_counties["Count_Tot"] = 1

In [ ]:
grouped = remaining_counties.groupby("UNIQUE_ID").sum()

In [ ]:
grouped.reset_index(inplace = True, drop = False)

In [ ]:
file_hold[file_hold["COUNTYFP"].isin([73])].to_csv("./last_few_counties_precincts.csv", index = False)

In [ ]:
grouped.to_csv("./last_few_counties_vf.csv", index = False)

In [ ]:
splits_to_make = {"Calhoun-:-1ST UN_ METHODIST-WEAVER":{"From":["Calhoun-:-ANGEL FIRE STATION"],
                                       "Blocks":["010150018023001","010150018023000","010150018012012"]},
"Henry-:-GREATER SHILOH BAPT_":{"From":["Henry-:-HEADLAND VTG CTR_ 1"],
                                "Blocks":["010670305001006",
                                         "010670305001015",
                                         "010670305001016",
                                         "010670305001017",
                                         "010670305001018",
                                         "010670305001019",
                                         "010670305001020",
                                         "010670305001021",
                                         "010670305001023",
                                         "010670305001024",
                                         "010670305001025",
                                         "010670305001026",
                                         "010670305001027",
                                         "010670305001028",
                                         "010670305001029",
                                         "010670305001031",
                                         "010670305001034",
                                         "010670305001035",
                                         "010670305001127"]},
"Jackson-:-WOODVILLE BOB JONES COMM":{"From":["Jackson-:-PLEASANT GROVES COMM CTR_"],
                                   "Blocks":["010719505001020",
                                            "010719505001023",
                                            "010719505001024",
                                            "010719505001044",
                                            "010719505001045",
                                            "010719505001046",
                                            "010719505001106",
                                            "010719505002059"]},
"Macon-:-SOJOURNER COMM CTR":{"From":["Macon-:-NATIONAL GUARD ARMORY"],
                           "Blocks":["010872319001007",
                                    "010872319001014",
                                    "010872319001015",
                                    "010872319001016",
                                    "010872319001017",
                                    "010872319001018",
                                    "010872319001019",
                                    "010872319001025",
                                    "010872319001026",
                                    "010872319001027",
                                    "010872317001027",
                                    "010872319001028",
                                    "010872319001029",
                                    "010872317001029",
                                    "010872319001030",
                                    "010872317001030",
                                    "010872319001031",
                                    "010872317001031",
                                    "010872317001032",
                                    "010872317001033",]},
"Macon-:-TUSKEGEE MUNICIAPL CMPLX":{"From":["Macon-:-NATIONAL GUARD ARMORY"],
                               "Blocks":["010872320001024",
                                        "010872320001025",
                                        "010872320001028",
                                        "010872320001029",
                                        "010872320001030",
                                        "010872320001035",
                                        "010872320001042",
                                        "010872320001043",
                                        "010872320001044"]},
"Lawrence-:-HILLSBORO CITY HALL 1-1":{"From":["Lawrence-:-CHALYBEATE FIRE DEPT17-1"],
                           "Blocks":["010799795022000",
                                    "010799795022001",
                                    "010799795022002",
                                    "010799795022003",
                                    "010799795022004",
                                    "010799795022006",
                                    "010799795022007",
                                    "010799795022009",
                                    "010799795022010",
                                    "010799795022016",
                                    "010799795022017",
                                    "010799795022018",
                                    "010799795022043"]},
"Lawrence-:-FREEMAN TAB 26-1":{"From":["Lawrence-:-BOARD OF ED 25-1","Lawrence-:-LAWRENCE HEALTH DEPT19-1"],
                               "Blocks":["010799794003045",
                                        "010799794004013",
                                        "010799794004015",
                                        "010799794004016",
                                        "010799794004017",
                                        "010799794004018"]},
"Lawrence-:-LAWRENCE HEALTH DEPT19-1":{"From":["Lawrence-:-BOARD OF ED 25-1"],
                                       "Blocks":["010799794004014","010799794004021","010799794004022"]},
"Lawrence-:-HATTON CH OF CHRIST 13-1":{"From":["Lawrence-:-MOULTON CITY HALL 24-1"],
                                    "Blocks":["010799798003015",
                                              "010799798003017",
                                             "010799798003018",
                                             "010799798003041",
                                             "010799798003042",
                                             "010799798003043",
                                             "010799798003076"]},
"Lawrence-:-BOARD OF ED 25-1":{"From":["Lawrence-:-LAWRENCE HEALTH DEPT19-1"],
                                "Blocks":["010799798003073",
                                         "010799798003072"]},
"Lawrence-:-MCCARLEY CTR_ 6-1":{"From":["Lawrence-:-PILGRAM MISS_ BAPT_ 18-1"],
                            "Blocks":["010799795021002",
                                     "010799795021000",
                                     "010799795021001",
                                     "010799795021006",
                                     "010799795021007"]},
"Bibb-:-SIX MILE COMM_ CTR_":{"From":["Bibb-:-ALTERNATIVE SCHOOL"],
                              "Blocks":["010070100012000",
                                       "010070100012001",
                                       "010070100012002",
                                       "010070100062008",
                                       "010070100012009",
                                       "010070100062009",
                                       "010070100012010",
                                       "010070100012011",
                                       "010070100082011",
                                       "010070100012012",
                                       "010070100082012",
                                       "010070100012013",
                                       "010070100062013",
                                       "010070100062014",
                                       "010070100012014",
                                       "010070100012015",
                                       "010070100062015",
                                       "010070100062016",
                                       "010070100062017",
                                       "010070100062018",
                                       "010070100062019",
                                       "010070100062021",
                                       "010070100062022",
                                       "010070100062023",
                                       "010070100062024",
                                       "010070100062025",
                                       "010070100062026",
                                       "010070100062027",
                                       "010070100062028",
                                       "010070100062029",
                                       "010070100012030",
                                       "010070100062030",
                                       "010070100012033",
                                       "010070100012036",
                                       "010070100012046",
                                       "010070100012047",
                                       "010070100012048",
                                       "010070100052078",
                                       "010070100052079"]},
"Bibb-:-ALTERNATIVE SCHOOL":{"From":["Bibb-:-GREENPOND FIRE DEPT_"],
                              "Blocks":['010070100053093',
                                '010070100051031',
                                '010070100053095',
                                '010070100053079',
                                '010070100051025',
                                '010070100051030',
                                '010070100053094',
                                '010070100053080',
                                '010070100051028',
                                '010070100011004',
                                '010070100011000',
                                '010070100053035',
                                '010070100053046',
                                '010070100053044',
                                '010070100053034',
                                '010070100011001',
                                '010070100053047',
                                '010070100051029',
                                '010070100011002',
                                '010070100053048',
                                '010070100051026',
                                '010070100053041',
                                '010070100051024',
                                '010070100011003',
                                '010070100053082',
                                '010070100051033',
                                '010070100053043',
                                '010070100053091',
                                '010070100053024',
                                '010070100053045',
                                '010070100053092',
                                '010070100051032',
                                '010070100053096',
                                '010070100053090',
                                '010070100011005',
                                '010070100053081',
                                '010070100053036',]},
"Pike-:-TROY PUBLIC LIBRARY":{"From":["Pike-:-TROY REC CTR"],
                              "Blocks":['011091889004006',
'011091889004004',
'011091889004009',
'011091889004011',
'011091889004012',
'011091889004008',
'011091889004022',
'011091889004016',
'011091889004013',
'011091889004005',
'011091889004018',
'011091889004007',
'011091889004015',
'011091889004010',]},
"Pike-:-SCOTT FARMS BLDG":{"From":["Pike-:-FIRST BAPT CHURCH", "Pike-:-PIKE CO COURTHOUSE"],
                              "Blocks":['011091890032028',
'011091892001010',
'011091892001008',
'011091892001009',
'011091890032007',
'011091890032006']}}

In [ ]:
# {"Calhoun-:-1ST UN_ METHODIST-WEAVER":{"From":["Calhoun-:-ANGEL FIRE STATION"],
#                                        "Blocks":["010150018023001","010150018023000","010150018012012"]}}
# {"Henry-:-GREATER SHILOH BAPT_":{"From":["Henry-:-HEADLAND VTG CTR_ 1"],
#                                 "Blocks":["010670305001006",
#                                          "010670305001015",
#                                          "010670305001016",
#                                          "010670305001017",
#                                          "010670305001018",
#                                          "010670305001019",
#                                          "010670305001020",
#                                          "010670305001021",
#                                          "010670305001023",
#                                          "010670305001024",
#                                          "010670305001025",
#                                          "010670305001026",
#                                          "010670305001027",
#                                          "010670305001028",
#                                          "010670305001029",
#                                          "010670305001031",
#                                          "010670305001034",
#                                          "010670305001035",
#                                          "010670305001127"]}}
# {"Jackson-:-WOODVILLE BOB JONES COMM":{"From":["Jackson-:-PLEASANT GROVES COMM CTR_"],
#                                    "Blocks":["010719505001020",
#                                             "010719505001023",
#                                             "010719505001024",
#                                             "010719505001044",
#                                             "010719505001045",
#                                             "010719505001046",
#                                             "010719505001106",
#                                             "010719505002059"]}}
# {"Macon-:-SOJOURNER COMM CTR":{"From":["Macon-:-NATIONAL GUARD ARMORY"],
#                            "Blocks":["010872319001007",
#                                     "010872319001014",
#                                     "010872319001015",
#                                     "010872319001016",
#                                     "010872319001017",
#                                     "010872319001018",
#                                     "010872319001019",
#                                     "010872319001025",
#                                     "010872319001026",
#                                     "010872319001027",
#                                     "010872317001027",
#                                     "010872319001028",
#                                     "010872319001029",
#                                     "010872317001029",
#                                     "010872319001030",
#                                     "010872317001030",
#                                     "010872319001031",
#                                     "010872317001031",
#                                     "010872317001032",
#                                     "010872317001033",]}}
# {"Macon-:-TUSKEGEE MUNICIAPL CMPLX":{"From":["Macon-:-NATIONAL GUARD ARMORY"],
#                                "Blocks":["010872320001024",
#                                         "010872320001025",
#                                         "010872320001028",
#                                         "010872320001029",
#                                         "010872320001030",
#                                         "010872320001035",
#                                         "010872320001042",
#                                         "010872320001043",
#                                         "010872320001044"]}}
# {"Lawrence-:-HILLSBORO CITY HALL 1-1":{"From":["Lawrence-:-CHALYBEATE FIRE DEPT17-1"],
#                            "Blocks":["010799795022000",
#                                     "010799795022001",
#                                     "010799795022002",
#                                     "010799795022003",
#                                     "010799795022004",
#                                     "010799795022006",
#                                     "010799795022007",
#                                     "010799795022009",
#                                     "010799795022010",
#                                     "010799795022016",
#                                     "010799795022017",
#                                     "010799795022018",
#                                     "010799795022043"]}}
# {"Lawrence-:-FREEMAN TAB 26-1":{"From":["Lawrence-:-BOARD OF ED 25-1","Lawrence-:-LAWRENCE HEALTH DEPT19-1"],
#                                "Blocks":["010799794003045",
#                                         "010799794004013",
#                                         "010799794004015",
#                                         "010799794004016",
#                                         "010799794004017",
#                                         "010799794004018"]}}
# {"Lawrence-:-LAWRENCE HEALTH DEPT19-1":{"From":["Lawrence-:-BOARD OF ED 25-1"],
#                                        "Blocks":["010799794004014","010799794004021","010799794004022"]}}
# {"Lawrence-:-HATTON CH OF CHRIST 13-1":{"From":["Lawrence-:-MOULTON CITY HALL 24-1"],
#                                     "Blocks":["010799798003015",
#                                               "010799798003017",
#                                              "010799798003018",
#                                              "010799798003041",
#                                              "010799798003042",
#                                              "010799798003043",
#                                              "010799798003076"]}}
# {"Lawrence-:-BOARD OF ED 25-1":{"From":["Lawrence-:-LAWRENCE HEALTH DEPT19-1"],
#                                 "Blocks":["010799798003073",
#                                          "010799798003072"]}}
# {"Lawrence-:-MCCARLEY CTR_ 6-1":{"From":["Lawrence-:-PILGRAM MISS_ BAPT_ 18-1"],
#                             "Blocks":["010799795021002",
#                                      "010799795021000",
#                                      "010799795021001",
#                                      "010799795021006",
#                                      "010799795021007"]}}
# {"Bibb-:-SIX MILE COMM_ CTR_":{"From":["Bibb-:-ALTERNATIVE SCHOOL"],
#                               "Blocks":["010070100012000",
#                                        "010070100012001",
#                                        "010070100012002",
#                                        "010070100062008",
#                                        "010070100012009",
#                                        "010070100062009",
#                                        "010070100012010",
#                                        "010070100012011",
#                                        "010070100082011",
#                                        "010070100012012",
#                                        "010070100082012",
#                                        "010070100012013",
#                                        "010070100062013",
#                                        "010070100062014",
#                                        "010070100012014",
#                                        "010070100012015",
#                                        "010070100062015",
#                                        "010070100062016",
#                                        "010070100062017",
#                                        "010070100062018",
#                                        "010070100062019",
#                                        "010070100062021",
#                                        "010070100062022",
#                                        "010070100062023",
#                                        "010070100062024",
#                                        "010070100062025",
#                                        "010070100062026",
#                                        "010070100062027",
#                                        "010070100062028",
#                                        "010070100062029",
#                                        "010070100012030",
#                                        "010070100062030",
#                                        "010070100012033",
#                                        "010070100012036",
#                                        "010070100012046",
#                                        "010070100012047",
#                                        "010070100012048",
#                                        "010070100052078",
#                                        "010070100052079"]}}
# {"Bibb-:-ALTERNATIVE SCHOOL":{"From":["Bibb-:-GREENPOND FIRE DEPT_"],
#                               "Blocks":['010070100053093',
#                                 '010070100051031',
#                                 '010070100053095',
#                                 '010070100053079',
#                                 '010070100051025',
#                                 '010070100051030',
#                                 '010070100053094',
#                                 '010070100053080',
#                                 '010070100051028',
#                                 '010070100011004',
#                                 '010070100011000',
#                                 '010070100053035',
#                                 '010070100053046',
#                                 '010070100053044',
#                                 '010070100053034',
#                                 '010070100011001',
#                                 '010070100053047',
#                                 '010070100051029',
#                                 '010070100011002',
#                                 '010070100053048',
#                                 '010070100051026',
#                                 '010070100053041',
#                                 '010070100051024',
#                                 '010070100011003',
#                                 '010070100053082',
#                                 '010070100051033',
#                                 '010070100053043',
#                                 '010070100053091',
#                                 '010070100053024',
#                                 '010070100053045',
#                                 '010070100053092',
#                                 '010070100051032',
#                                 '010070100053096',
#                                 '010070100053090',
#                                 '010070100011005',
#                                 '010070100053081',
#                                 '010070100053036',]}}
# {"Pike-:-TROY PUBLIC LIBRARY":{"From":["Pike-:-TROY REC CTR"],
#                               "Blocks":['011091889004006',
# '011091889004004',
# '011091889004009',
# '011091889004011',
# '011091889004012',
# '011091889004008',
# '011091889004022',
# '011091889004016',
# '011091889004013',
# '011091889004005',
# '011091889004018',
# '011091889004007',
# '011091889004015',
# '011091889004010',]}}
# {"Pike-:-SCOTT FARMS BLDG":{"From":["Pike-:-FIRST BAPT CHURCH", "Pike-:-PIKE CO COURTHOUSE"],
#                               "Blocks":['011091890032028',
# '011091892001010',
# '011091892001008',
# '011091892001009',
# '011091890032007',
# '011091890032006',]}}

In [ ]:
madison = gp.read_file("/Users/peterhorton/Downloads/2022 Precincts/2022_Precincts.shp")

In [ ]:
madison.plot()

## Look Into Other Counties

In [ ]:
remaining_counties = [i for i in file_hold["County"].unique() if i not in full_list]
print(len(remaining_counties))

In [ ]:
holder

In [ ]:
file_hold = file_hold.to_crs(4326)

In [ ]:
file_hold[file_hold["UNIQUE_ID"]=="Escambia-:-BETHEL ROBERTS"].plot()

In [ ]:
remaining_counties.sort()

In [ ]:
for county in list(file_hold["County"].unique()):
    if not os.path.exists("./vf_map_checks_no_match/"+county):
        os.mkdir("./vf_map_checks_no_match/"+county)

    for key,row in file_hold[file_hold["County"]==county].iterrows():
        fips = row["COUNTYFP"]
        #print(row["UNIQUE_ID"])
        prec = file_hold[file_hold["UNIQUE_ID"]==row["UNIQUE_ID"]].copy(deep = True)
        ax = file_hold[file_hold["UNIQUE_ID"]==row["UNIQUE_ID"]].boundary.plot(color = "red")
        ax.set_title(row["UNIQUE_ID"])

        intersect = gp.sjoin(prec, holder[(holder["co_fips"]==fips)&(holder["gen_vote"]=="Y")&~(holder["UNIQUE_ID"].isna())], how = "inner")

        if intersect.shape[0] != 0:
            holder[(holder["co_fips"]==fips)&(holder["UNIQUE_ID"].isin(list(intersect["UNIQUE_ID_right"])))].plot(ax = ax, markersize = 1, column = "UNIQUE_ID", legend = True, legend_kwds = {'loc':(1,-.25)})
            
#             xlim = ([prec.total_bounds[0],  prec.total_bounds[2]])
#             ylim = ([prec.total_bounds[1],  prec.total_bounds[3]])

            ax.set_xlim(xlim)
            ax.set_ylim(ylim)
            
        ax.figure.savefig("./vf_map_checks_no_match/"+county+"/"+row["UNIQUE_ID"].replace("/","-")+".jpg", bbox_inches='tight')               

In [ ]:
# #for county in counties_w_data:
# for county in counties_w_data:

#     county_data = file_hold[file_hold["County"]==county].copy(deep = True)
#     fips = county_data["COUNTYFP"].unique()[0]
#     county_vf = holder[holder["co_fips"]==fips]
#     county_join = gp.sjoin(county_data, county_vf, how = "right")
#     print(county_join)

#     if not os.path.exists("./vf_map_checks/"+county):
#         os.mkdir("./vf_map_checks/"+county)
#     for key,row in vf_correspondence[vf_correspondence["County"]==county].iterrows():
# #     print(row["VF_ID"])
# #     print(row["ELEC_22_ID"])
        

#         prec = file_hold[file_hold["UNIQUE_ID"]==row["ELEC_22_ID"]].copy(deep = True)
#         ax = file_hold[file_hold["UNIQUE_ID"]==row["ELEC_22_ID"]].boundary.plot(figsize = (10,10))
#         holder[holder["UNIQUE_ID"]==row["VF_ID"]].plot(ax = ax, color = "red")
#         fips = prec["COUNTYFP"].values[0]
        
#         intersect = county_join[(county_join["UNIQUE_ID_x"]==row["ELEC_22_ID"])&(county_join["UNIQUE_ID_y"]!=row["VF_ID"])]
             
#         if intersect.shape[0] != 0:
#             intersect.plot(ax = ax, column = "ELEC_PREC_MATCH", legend = True, legend_kwds = {'loc':(1,0)})
        
        
        
        
#         ax.set_title(row["ELEC_22_ID"])
#         ax.figure.savefig("./vf_map_checks/"+county+"/"+row["ELEC_22_ID"].replace("/","-")+".jpg", bbox_inches = "tight")

In [ ]:
#099-1203-MCMIDLONES CC - is this Monroe unassigned?